To run this Application write this in your terminal:

```py -3.10 -m venv myenv```

```.\myenv\Scripts\Activate```

select the kernal to be myenv (Python 3.10.0)

```py --version``` insure this version is 3.10.0


In [39]:
!py --version

Python 3.10.0


- **this cell could take some time to run**

In [40]:
import subprocess
import importlib
import os
import json
from pathlib import Path
    
# Define the libraries and their pip install commands
libraries = {
    "tensorflow": "pip install tensorflow",
    "tensorflow-hub": "pip install tensorflow-hub",
    "numpy": "pip install numpy",
    "matplotlib": "pip install matplotlib",
    "pandas": "pip install pandas",
    "cv2": "pip install opencv-python", 
    "keras": "pip install keras",
}

for library, command in libraries.items():
    try:
        importlib.import_module(library)
        print(f"{library} is already installed.")
    except ImportError:
        print(f"{library} not found. Installing...")
        try:
            subprocess.run(command.split(), check=True)
            print(f"{library} installed successfully.")
        except subprocess.CalledProcessError as e:
            print(f"Failed to install {library}. Error: {e}")

try:
    import tensorflow as tf
    import tensorflow_hub as hub
    from keras import models, layers, applications
    import numpy as np
    import cv2
    
    print("All required libraries have been imported successfully!")
except ImportError as e:
    print(f"Error importing libraries: {e}")


tensorflow is already installed.
tensorflow-hub not found. Installing...
tensorflow-hub installed successfully.
numpy is already installed.
matplotlib is already installed.
pandas is already installed.
cv2 is already installed.
keras is already installed.
All required libraries have been imported successfully!


In [41]:
def video_to_frames(video_path, output_folder, frame_rate=1):
    """
    Extracts frames from a video at a specified frame rate.

    Args:
        video_path (str): Path to the video file.
        output_folder (str): Folder to save the extracted frames.
        frame_rate (int): Number of frames to save per second of video.

    Returns:
        None
    """
    if frame_rate <= 0:
        print("Error: Frame rate must be a positive integer.")
        return

    # Create the output folder if it doesn't exist
    os.makedirs(output_folder, exist_ok=True)

    # Open the video file
    video = cv2.VideoCapture(video_path)
    if not video.isOpened():
        print(f"Error: Could not open video file '{video_path}'.")
        return

    # Get the video's frame rate and total number of frames
    video_fps = int(video.get(cv2.CAP_PROP_FPS))
    if video_fps <= 0:
        print("Error: Invalid video FPS. Please check the video file.")
        video.release()
        return

    if frame_rate > video_fps:
        print(f"Warning: Frame rate ({frame_rate}) exceeds video FPS ({video_fps}). Adjusting to match video FPS.")
        frame_rate = video_fps

    frame_count = int(video.get(cv2.CAP_PROP_FRAME_COUNT))
    print(f"Video FPS: {video_fps}, Total Frames: {frame_count}")

    # Calculate the frame interval for the desired frame rate
    frame_interval = video_fps // frame_rate

    # Extract and save frames
    frame_idx = 0
    saved_frame_idx = 0
    while True:
        ret, frame = video.read()
        if not ret:
            break  # End of video

        # Save the frame at the specified interval
        if frame_idx % frame_interval == 0:
            frame_filename = os.path.join(output_folder, f"frame_{saved_frame_idx:04d}.jpg")
            cv2.imwrite(frame_filename, frame)
            saved_frame_idx += 1

        frame_idx += 1

    # Release the video capture object
    video.release()

    if saved_frame_idx == 0:
        print(f"No frames were saved. Adjust the frame rate and try again.")
    else:
        print(f"{saved_frame_idx} frames saved to '{output_folder}'.")
        
    return saved_frame_idx

In [42]:
target_extraction_folders = [
    "workspace/good_pose",
    "workspace/bad_pose",
    "workspace/cant_determine"
]


def process_session(session_path, label, output_dir, frame_rate=1):
    """
    Processes a single session folder:
      1. Finds video and JSON files.
      2. Extracts frames using 'video_to_frames' at the desired rate.
      3. Reads gyroscope/accelerometer data from JSON (no timestamps).
      4. Maps each extracted frame to a sensor reading by index.
    """
    # Identify the video & JSON inside this session folder
    video_path = None
    json_path = None

    for file_name in os.listdir(session_path):
        file_path = os.path.join(session_path, file_name)
        lower = file_name.lower()
        if lower.endswith(('.mp4')):
            video_path = file_path
        elif lower.endswith('.json'):
            json_path = file_path

    if not video_path or not os.path.exists(video_path):
        print(f"No valid video found in: {session_path}")
        return
    if not json_path or not os.path.exists(json_path):
        print(f"No valid JSON file found in: {session_path}")
        return

    # Read JSON
    with open(json_path, 'r') as f:
        sensor_data = json.load(f)

    # Potentially these arrays could be different lengths. 
    # Example:
    # sensor_data = {
    #   "gyroscopeData": [ {x:..., y:..., z:...}, ... ],
    #   "accelerometerData": [ {x:..., y:..., z:...}, ... ]
    # }
    gyro_list = sensor_data.get("gyroscopeData", [])
    accel_list = sensor_data.get("accelerometerData", [])
    
    # For example, if we want to label frames by 
    # “average magnitude” or something else, we can compute it.
    # Here, we’ll just store the raw sensor data as we have no official posture label.

    # Make an output folder for the frames of this session
    # e.g. extracted_frames/good_pose/session_01/
    session_name = Path(session_path).name
    session_output = os.path.join(output_dir, label, session_name)
    os.makedirs(session_output, exist_ok=True)

    # 1. Extract frames
    saved_frames = video_to_frames(
        video_path=video_path,
        output_folder=session_output,
        frame_rate=frame_rate
    )

    # 2. If you want to "map" each extracted frame to a sensor reading:
    #    Let's say we have M = saved_frames, N = len(gyro_list).
    #    We'll create a small CSV or rename frames to reflect the sensor reading.

    if saved_frames == 0:
        print("No frames were extracted; skipping sensor mapping.")
        return

    # M: number of frames saved, N: number of sensor readings
    M = saved_frames
    N = len(gyro_list)
    L = len(accel_list)  
    
    
    if N == 0:
        print("No sensor readings found in JSON; skipping sensor mapping.")
        return
    
    if M == 0:
        print("No frames were extracted; skipping sensor mapping.")
        return

    # Let’s rename frames or create a CSV that lines them up:
    # For each frame i in [0..M-1], we pick sensor_idx = floor(i / M * N)
    # Then store or rename the frame accordingly.

    # Example: rename "frame_0000.jpg" => "frame_0000_idx_0.jpg"
    # and so forth. Or create a CSV with the sensor data.

    frame_files = sorted([
        f for f in os.listdir(session_output) if f.lower().endswith('.jpg')
    ])

    # If you'd prefer to store mappings in a CSV:
    csv_path = os.path.join(session_output, "frame_sensor_mapping.csv")
    with open(csv_path, 'w') as csv_file:
        csv_file.write("frame_file,sensor_idx,gyro_x,gyro_y,gyro_z\n")
        for i, frame_file in enumerate(frame_files):
            gyro_idx = int((i / M) * N)  # integer ratio mapping
            accel_idx = int((i / M) * L)
            
            # Grab the sensor reading from gyro_list
            gyro_data = gyro_list[gyro_idx]  # { x: ?, y: ?, z: ? }
            accel_data = accel_list[accel_idx]
            
            # Write to CSV
            # csv_file.write(
            #     f"{frame_file},{gyro_idx},"
            #     f"{gyro_data.get('x', 0)},{gyro_data.get('y', 0)},{gyro_data.get('z', 0)}\n"
            # )
            csv_file.write(
                f"{frame_file},{gyro_idx},"
                f"{gyro_data.get('x', 0)},{gyro_data.get('y', 0)},{gyro_data.get('z', 0)},"
                f"{accel_data.get('x', 0)},{accel_data.get('y', 0)},{accel_data.get('z', 0)}\n"
            )

            # (Optionally) rename the frame file with sensor_idx if you want
            old_path = os.path.join(session_output, frame_file)
            base, ext = os.path.splitext(frame_file)
            new_filename = f"{base}_idx_{gyro_idx}{ext}"
            new_path = os.path.join(session_output, new_filename)
            os.rename(old_path, new_path)

    print(f"Frame-to-sensor mapping complete for: {session_path}")


In [ ]:
output_dir = "extracted_frames"
# frame_rate = 30
frame_rate = 1 # for demonstration purposes

for label in target_extraction_folders:
    label_path = Path(label)
    if not label_path.exists():
        print(f"Label folder not found: {label_path}")
        continue

    # Each subfolder in e.g. "good_pose" is a session
    for session_name in os.listdir(label_path):
        session_path = os.path.join(label_path, session_name)
        if not os.path.isdir(session_path):
            continue

        try:
            process_session(
                session_path=session_path,
                label=label,
                output_dir=output_dir,
                frame_rate=frame_rate
            )
        except Exception as e:
            print(f"Error processing session '{session_path}': {e}")


Video FPS: 29, Total Frames: 751
26 frames saved to 'extracted_frames\workspace/good_pose\1'.
Frame-to-sensor mapping complete for: workspace\good_pose\1
Video FPS: 29, Total Frames: 667
23 frames saved to 'extracted_frames\workspace/good_pose\11'.
No sensor readings found in JSON; skipping sensor mapping.
Video FPS: 29, Total Frames: 1246
43 frames saved to 'extracted_frames\workspace/good_pose\15'.
Frame-to-sensor mapping complete for: workspace\good_pose\15
Video FPS: 29, Total Frames: 826
29 frames saved to 'extracted_frames\workspace/good_pose\17'.
Frame-to-sensor mapping complete for: workspace\good_pose\17
Video FPS: 29, Total Frames: 549
19 frames saved to 'extracted_frames\workspace/good_pose\2'.
Frame-to-sensor mapping complete for: workspace\good_pose\2
Video FPS: 29, Total Frames: 946
33 frames saved to 'extracted_frames\workspace/good_pose\21'.
Frame-to-sensor mapping complete for: workspace\good_pose\21
Video FPS: 30, Total Frames: 56
2 frames saved to 'extracted_frames\w

In [38]:
# Image input
image_input = layers.Input(shape=(224, 224, 3))
base_model = applications.MobileNetV2(weights='imagenet', include_top=False, input_tensor=image_input)
image_features = layers.GlobalAveragePooling2D()(base_model.output)

# Sensor input
sensor_input = layers.Input(shape=(6,))  # 3 gyroscope + 3 accelerometer features
sensor_features = layers.Dense(64, activation='relu')(sensor_input)

# Combine features
combined = layers.concatenate([image_features, sensor_features])
combined = layers.Dense(128, activation='relu')(combined)
output = layers.Dense(2, activation='softmax')(combined)

model = models.Model(inputs=[image_input, sensor_input], outputs=output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

C:\Users\obada\AppData\Local\Temp\ipykernel_13032\2539727540.py:3: UserWarning: `input_shape` is undefined or non-square, or `rows` is not in [96, 128, 160, 192, 224]. Weights for input shape (224, 224) will be loaded as the default.
  base_model = applications.MobileNetV2(weights='imagenet', include_top=False, input_tensor=image_input)


Preprocess Your Data: Extract frames from your videos and sync them with your sensor data. You might need to downsample or interpolate the sensor data to match the frame rate.
Prepare Your Dataset: Create labeled pairs of video frames and corresponding sensor data, with labels for good or bad posture.
Build the Model: Use MobileNet for feature extraction from video frames. Create a separate neural network for the sensor data.
Combine the Outputs: Merge the outputs from both networks and add a final classification layer.
Train the Model: Use your labeled dataset to train the combined model.


Feature extraction, 
build a separate network